<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>

# Instrumenting AWS Bedrock client with OpenInference and Phoenix

In this tutorial we will trace model calls to AWS Bedrock using OpenInference. The OpenInference Bedrock tracer instruments the Python `boto3` library, so all `invoke_model` calls will automatically generate traces that can be sent to Phoenix.

ℹ️ This notebook requires a valid AWS configuration and access to AWS Bedrock and the `claude-v2` model from Anthropic.

## 1. Install dependencies and set up OpenTelemetry tracer

First install dependencies

In [ ]:
!pip install arize-phoenix boto3 openinference-instrumentation-bedrock

Import libraries

In [1]:
import json
from urllib.parse import urljoin

import boto3
import phoenix as px
from openinference.instrumentation.bedrock import BedrockInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import ConsoleSpanExporter, SimpleSpanProcessor

Start a Pheonix server to collect traces. Be sure to view Phoenix in your browser to watch traces show up in Phoenix as they are collected.

In [2]:
px.launch_app()
session_url = px.active_session().url

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


Here we're configuring the OpenTelemetry tracer by adding two SpanProcessors. The first SpanProcessor will simply print all traces received from OpenInference instrumentation to the console. The second will export traces to Phoenix so they can be collected and viewed.

In [3]:
resource = Resource(attributes={})
tracer_provider = trace_sdk.TracerProvider(resource=resource)
span_console_exporter = ConsoleSpanExporter()
phoenix_otlp_endpoint = urljoin(session_url, "v1/traces")
phoenix_exporter = OTLPSpanExporter(endpoint=phoenix_otlp_endpoint)
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_console_exporter))
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=phoenix_exporter))
trace_api.set_tracer_provider(tracer_provider=tracer_provider)

## 2. Instrumenting Bedrock clients

Now, let's create a `boto3` session. This initiates a configured environment for interacting with AWS services. If you haven't yet configured `boto3` to use your credentials, please refer to the [official documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/configuration.html). Or, if you have the AWS CLI, run `aws configure` from your terminal.

In [4]:
session = boto3.session.Session()

Clients created using this session configuration are currently uninstrumented. We'll make one for comparison.

In [5]:
uninstrumented_client = session.client("bedrock-runtime")

Now we instrument Bedrock with our OpenInference instrumentor. All Bedrock clients created after this call will automatically produce traces when calling `invoke_model`.

In [6]:
BedrockInstrumentor().instrument()
instrumented_client = session.client("bedrock-runtime")

## 3. Calling the LLM and viewing OpenInference traces

Calling `invoke_model` using the `uninstrumented_client` will produce no traces, but will show the output from the LLM.

In [7]:
prompt = b'{"prompt": "Human: Hello there, how are you? Assistant:", "max_tokens_to_sample": 1024}'
response = uninstrumented_client.invoke_model(modelId="anthropic.claude-v2", body=prompt)
response_body = json.loads(response.get("body").read())
print(response_body["completion"])

 I'm doing well, thanks for asking!


LLM calls using the `instrumented_client` will print traces to the console! By configuring the `SpanProcessor` to export to a different OpenTelemetry collector, your OpenInference spans can be collected and analyzed to better understand the behavior of your LLM application.

In [8]:
response = instrumented_client.invoke_model(modelId="anthropic.claude-v2", body=prompt)
response_body = json.loads(response.get("body").read())
print(response_body["completion"])

{
    "name": "bedrock.invoke_model",
    "context": {
        "trace_id": "0x994497f3948acc4cf390658d6f43dd7c",
        "span_id": "0x3180fe54cacc8f9b",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": null,
    "start_time": "2024-02-15T15:52:13.188122Z",
    "end_time": "2024-02-15T15:52:14.664294Z",
    "status": {
        "status_code": "UNSET"
    },
    "attributes": {
        "llm.prompts": "Human: Hello there, how are you? Assistant:",
        "llm.invocation_parameters": "{\"max_tokens_to_sample\": 1024}",
        "llm.model_name": "anthropic.claude-v2",
        "message.content": " I'm doing well, thanks for asking!"
    },
    "events": [],
    "links": [],
    "resource": {
        "attributes": {},
        "schema_url": ""
    }
}
 I'm doing well, thanks for asking!


More information about our instrumentation integrations, OpenInerence can be found in our [documentation](https://docs.arize.com/phoenix/telemetry/instrumentation)